In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

ModuleNotFoundError: No module named 'lightgbm'

# 1.1 Read Train and Test Dataset

In [17]:
%%time
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
sample_df = pd.read_csv("../input/sample_submission.csv")

FileNotFoundError: File b'../input/train.csv' does not exist

# 1.2 VALUE COUNTS ON TARGET COLUMN

In [ ]:
train_df['target'].value_counts()

# 1.3 CHECK NULL DATA

In [ ]:
train_null = train_df.isnull().sum().sort_values(ascending=False)
test_null = test_df.isnull().sum().sort_values(ascending=False)
null_datacheck = pd.concat([train_null,test_null],axis=1,keys=['train_null','test_null'])
null_datacheck

# 2.1 CORRELATION

In [ ]:
corr = train_df.corr()
abs(corr['target']).sort_values(ascending=False)

# 3.1 CREATE MODEL AND COMPARE ACCURACY

In [ ]:
%%time
from sklearn.model_selection import train_test_split

y = train_df['target'].astype('float32')
X = train_df.values[:,2:].astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X,y)

## LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train,y_train)

In [ ]:
logreg_train_score = logreg.score(X_train,y_train)
logreg_test_score = logreg.score(X_test,y_test)

logreg_train_score, logreg_test_score

In [ ]:
X_test = test_dt.values[:,1:].astype('float32')
y_pred = logreg.predict(X_test)
sample_dt['target']=y_pred
sample_dt.to_csv("log_reg_submission.csv", index=False)

## GRADIENT BOOSTING

In [ ]:
%%time

from sklearn.ensemble import (GradientBoostingClassifier, 
                              GradientBoostingRegressor)
from sklearn.model_selection import GridSearchCV

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.05],
    "max_depth":[3,6],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.6, 1.0],
    "n_estimators":[10]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1)

clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))
print(clf.best_params_)

In [ ]:
X_test = test_dt.values[:,1:].astype('float32')
y_pred = gbrt.predict(X_test)
sample_dt['target']=y_pred
sample_dt.to_csv("gbrt_submission.csv", index=False)

## LIGHTGBM

In [ ]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

In [ ]:
param = {
        'num_leaves': 6,
        'max_bin': 63,
        'min_data_in_leaf': 45,
        'learning_rate': 0.01,
        'min_sum_hessian_in_leaf': 0.000446,
        'bagging_fraction': 0.55, 
        'bagging_freq': 5, 
        'max_depth': 14,
        'save_binary': True,
        'seed': 31452,
        'feature_fraction_seed': 31415,
         'feature_fraction': 0.51,
        'bagging_seed': 31415,
        'drop_seed': 31415,
        'data_random_seed': 31415,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

In [ ]:
folds = StratifiedKFold(n_splits=9, shuffle=True, random_state=31415)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 15000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 250)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))


In [ ]:
##submission
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions
sub_df.to_csv("lgb_submission.csv", index=False)